# 🦌 DINO Animal Detection - HSE Zapovednik

**Анализ изображений с фотоловушек с помощью модели DINO**

Этот ноутбук позволяет:
- Запустить детекцию животных на вашем датасете
- Извлечь метаданные (дата, время, температура) через OCR
- Рассчитать ROC AUC метрики (если есть аннотации)
- Сформировать отчёты в формате CSV

---

## 1. Настройка окружения

In [ ]:
# Проверка GPU
!nvidia-smi
print("\n" + "="*60)
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("="*60)

In [ ]:
# =====================================================
# КЛОНИРОВАНИЕ РЕПОЗИТОРИЯ
# =====================================================
# Замените YOUR_USERNAME на ваш GitHub username

!git clone https://github.com/YOUR_USERNAME/hse-zapovednik.git
%cd hse-zapovednik
print("\n[OK] Репозиторий склонирован")

In [ ]:
# Установка зависимостей
print("[SETUP] Установка зависимостей...")
%pip install -q timm pycocotools easyocr scikit-learn Pillow tqdm
print("[OK] Зависимости установлены")

In [ ]:
# Компиляция CUDA операций для DINO (Deformable Attention)
import os
os.chdir('DINO/models/dino/ops')
!python setup.py build install
os.chdir('../../../..')
print("\n[OK] CUDA операции скомпилированы")

## 2. Загрузка checkpoint модели

Скачайте checkpoint с Яндекс.Диска / Google Drive и загрузите в Colab

In [ ]:
# Создаём папку для checkpoint
!mkdir -p output_dino

# =====================================================
# ВАРИАНТ 1: Загрузка файла вручную через интерфейс Colab
# =====================================================
from google.colab import files
print("Загрузите checkpoint.pth:")
uploaded = files.upload()

# Перемещаем файл
import shutil
for filename in uploaded.keys():
    shutil.move(filename, 'output_dino/checkpoint.pth')
    print(f"[OK] Checkpoint сохранён: output_dino/checkpoint.pth")

## 3. Загрузка датасета

**Поддерживаемые форматы:**

### COCO формат:
```
dataset/
├── annotations/
│   └── instances_val2017.json
└── val2017/
    └── *.jpg, *.png
```

### YOLO формат (Ultralytics):
```
dataset/
├── labels/
│   └── *.txt
└── images/
    └── *.jpg, *.png
```

In [ ]:
# Загрузка датасета (ZIP архив)
from google.colab import files
import zipfile

print("Загрузите ZIP архив с датасетом:")
uploaded = files.upload()

# Распаковываем
for filename in uploaded.keys():
    print(f"[INFO] Распаковка {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('dataset')
    print(f"[OK] Датасет распакован в папку 'dataset'")

# Показываем структуру
!ls -la dataset/

## 4. Запуск анализа

In [ ]:
# =====================================================
# НАСТРОЙКИ АНАЛИЗА - ИЗМЕНИТЕ ПОД СВОЙ ДАТАСЕТ
# =====================================================

# Путь к датасету
DATASET_PATH = "dataset"

# Формат датасета: "coco", "yolo", или "none" (только изображения)
FORMAT = "coco"

# Порог уверенности детекции (0.0 - 1.0)
THRESHOLD = 0.3

# Использовать OCR для извлечения метаданных
USE_OCR = True

print("="*60)
print("ПАРАМЕТРЫ АНАЛИЗА")
print("="*60)
print(f"Датасет:  {DATASET_PATH}")
print(f"Формат:   {FORMAT}")
print(f"Порог:    {THRESHOLD}")
print(f"OCR:      {'Да' if USE_OCR else 'Нет'}")
print("="*60)

In [ ]:
# Определяем пути к изображениям и аннотациям автоматически
import os

def find_paths(dataset_path, format_type):
    images_dir = None
    annotations = None
    
    if format_type == "coco":
        # Ищем папку с изображениями
        for subdir in ['val2017', 'images/val', 'images', '']:
            path = os.path.join(dataset_path, subdir) if subdir else dataset_path
            if os.path.exists(path) and any(f.endswith(('.jpg', '.png', '.jpeg')) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))):
                images_dir = path
                break
        
        # Ищем аннотации
        ann_dir = os.path.join(dataset_path, 'annotations')
        if os.path.exists(ann_dir):
            for ann_file in ['instances_val2017.json', 'instances_val.json']:
                ann_path = os.path.join(ann_dir, ann_file)
                if os.path.exists(ann_path):
                    annotations = ann_path
                    break
            if not annotations:
                json_files = [f for f in os.listdir(ann_dir) if f.endswith('.json')]
                if json_files:
                    annotations = os.path.join(ann_dir, json_files[0])
    
    elif format_type == "yolo":
        for subdir in ['images/val', 'images', '']:
            path = os.path.join(dataset_path, subdir) if subdir else dataset_path
            if os.path.exists(path):
                images_dir = path
                break
        
        for subdir in ['labels/val', 'labels']:
            path = os.path.join(dataset_path, subdir)
            if os.path.exists(path):
                annotations = path
                break
    else:
        images_dir = dataset_path
    
    return images_dir, annotations

images_dir, annotations = find_paths(DATASET_PATH, FORMAT)

print(f"[INFO] Папка с изображениями: {images_dir}")
print(f"[INFO] Аннотации: {annotations if annotations else 'не найдены'}")

if images_dir and os.path.exists(images_dir):
    img_count = len([f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    print(f"[INFO] Найдено изображений: {img_count}")

In [ ]:
# =====================================================
# ЗАПУСК АНАЛИЗА
# =====================================================

# Формируем команду
cmd = f"python run_analysis.py --checkpoint output_dino/checkpoint.pth --images_dir {images_dir} --threshold {THRESHOLD}"

if annotations:
    cmd += f" --annotations {annotations} --format {FORMAT}"

if USE_OCR:
    cmd += " --use_ocr"

print("="*60)
print("ЗАПУСК АНАЛИЗА")
print("="*60)
print(f"Команда: {cmd}")
print("="*60 + "\n")

# Запускаем
!{cmd}

## 5. Просмотр результатов

In [ ]:
import pandas as pd
import json
import os

results_dir = "results"

if os.path.exists(results_dir):
    print("\n" + "="*60)
    print("РЕЗУЛЬТАТЫ АНАЛИЗА")
    print("="*60)
    
    # Таблица 1: Сводка по видам
    summary_file = os.path.join(results_dir, "summary_by_species.csv")
    if os.path.exists(summary_file):
        print("\n📊 ТАБЛИЦА 1: Сводка по видам животных\n")
        df_summary = pd.read_csv(summary_file)
        display(df_summary)
    
    # Таблица 2: Детальный лог
    detailed_file = os.path.join(results_dir, "detailed_detections.csv")
    if os.path.exists(detailed_file):
        print("\n📋 ТАБЛИЦА 2: Детальный лог детекций (первые 30 записей)\n")
        df_detailed = pd.read_csv(detailed_file)
        display(df_detailed.head(30))
        print(f"\n... всего {len(df_detailed)} записей")
    
    # ROC AUC метрики
    json_file = os.path.join(results_dir, "analysis_results.json")
    if os.path.exists(json_file):
        with open(json_file, 'r', encoding='utf-8') as f:
            results = json.load(f)
        
        if 'roc_auc' in results and results['roc_auc']:
            print("\n📈 ROC AUC МЕТРИКИ\n")
            roc_data = results['roc_auc']
            
            if 'mean_roc_auc' in roc_data:
                print(f"  Средний ROC AUC: {roc_data['mean_roc_auc']:.4f}")
            
            if 'per_class' in roc_data:
                print("\n  По классам:")
                for class_name, auc_val in sorted(roc_data['per_class'].items(), key=lambda x: x[1], reverse=True):
                    print(f"    {class_name}: {auc_val:.4f}")
else:
    print("[ERROR] Папка results не найдена. Сначала запустите анализ.")

In [ ]:
# Скачивание результатов
from google.colab import files
import shutil

if os.path.exists('results'):
    shutil.make_archive('analysis_results', 'zip', 'results')
    files.download('analysis_results.zip')
    print("[OK] Архив с результатами скачивается")
else:
    print("[ERROR] Нет результатов для скачивания")

---

## 📝 Классы животных

Модель обучена распознавать следующие виды:

| ID | Русское название | English |
|:--:|:-----------------|:--------|
| 0 | Росомаха | Wolverine |
| 1 | Выдра | Otter |
| 2 | Рысь | Lynx |
| 3 | Горностай | Ermine |
| 4 | Волк | Wolf |
| 5 | Медведь | Bear |
| 6 | Лось | Moose |
| 7 | Олень | Deer |
| 8 | Лисица | Fox |
| 9 | Кабан | Wild boar |
| 10 | Заяц-беляк | Mountain hare |
| 11 | Белка | Squirrel |
| 12 | Глухарь | Capercaillie |
| 13 | Куница | Marten |
| 14 | Енотовидная собака | Raccoon dog |
| 15 | Норка | Mink |